In [ ]:
import yfinance as yf
import pandas as pd
#importando dash plotly
from dash import Dash, html, dash_table, dcc, Input, Output
import plotly.express as px
import matplotlib.pyplot as plt


#carregando lista de simbolos
# lista_de_simbolos = pd.read_csv('symbols_valid_meta.csv')
# display(lista_de_simbolos.head(2))
###para testes ####
lista_de_simbolos = ['MSFT34.SA', 'PETR3.SA', 'AAPL34.SA']

#definindo Periodos
PERIODOS = ['5y','2y','1y','9mo','6mo','3mo','1mo','5d','1d'] #len 9


In [ ]:

#func para criar dataframe com todas as acoes nescessarias
def creat_df(acoes:list, periodo:str = '3mo'):
    df_final = pd.DataFrame()
    dados = pd.DataFrame()
    for acao in acoes:
        dados = pd.DataFrame(yf.Ticker(acao).history(period=periodo)) #carrega o historico
        dados['Titulo'] = acao
        df_final = pd.concat([df_final,dados])
    return df_final.sort_values(by='Date')

dados = creat_df(lista_de_simbolos)
dados.head(5)

In [ ]:
#desenhando desenvolvimento acao
import plotly.graph_objects as go
def fig_candlestick(dados):
    fig = px.line(data_frame=dados,x=dados.index ,y=['Open'], color='Titulo')
    
    return fig


fig_candlestick(dados)

# Dash app

In [ ]:
#inicializar app
app = Dash(__name__)

#definindo layout app
app.layout = html.Div([
    html.H1('Acompanhamento de Ações' ,style={'backgroundColor':'white','text-align': 'center'} ),
    html.Hr(),
    html.H3('Escolha a acao:',style={'backgroundColor':'white'}),
    dcc.Dropdown(id='escolha-de-acoes', options=lista_de_simbolos, multi=True, value=lista_de_simbolos[:2]),
    dcc.Graph(figure=fig_candlestick(dados)),
    
])

if __name__ == '__main__':
    app.run(debug=True)